In [133]:
#import necessary libraries
import numpy as np
import math
import pickle
import simplejson
import time
import pdb

In [134]:
#graph can be defined using an adjacency list
class node:
    def __init__(self,vert_id=None,weight=None,nex=None):
        self.vert_id=vert_id
        self.weight=weight
        self.nex=nex
    def __str__(self):
        return('Vert ID: '+ str(self.vert_id) +' Weight : '+ str(self.weight))

class l_list(node):
    def __init__(self,node):
        self.head=node

Now let us add some edges to this adjacency list at random with average degree of each vertex as 6 and random positive weights. 
I have made sure to avoid multiple edges. This is for the first graph G1.

In [135]:
def graph_gen1():
    #Now let us create an adjacency list with all the vertices in the graph
    adj_list1=[]
    for i in range(5000):
        head=node(i)
        adj_list1.append(head)
    verts=list(np.arange(5000))
    
    edges1=[]
    for v in range (0,5000):    
        degree_v=np.random.randint(2,8)
        p=adj_list1[v]
        while(p.nex!=None):
            p=p.nex
        vert_ids=np.random.choice(verts,min(degree_v,len(verts)-1),replace=False)
        for edge in range(0,min(degree_v,len(verts)-1)):
            weight=np.random.randint(1,1000)
            w=vert_ids[edge]
            p.nex=node(w,weight)
            edges1.append([v,w,weight])
            p=p.nex

            q=adj_list1[w]
            while(q.nex!=None):
                q=q.nex
            q.nex=node(v,weight)
        #add one edge to next vertex to ensure graph is connected
        if v<4999:
            weight=np.random.randint(1,1000)
            p.nex=node(v+1,weight)
        verts.remove(v)
    return adj_list1,edges1

Now let us create one more graph G2 with edges assigned to around 20% of the remaining vertices. So it is clear that this graph is much more dense, there are far more number of edges. 

In [136]:
def graph_gen2():
    #Now let us create an adjacency list with all the vertices in the graph
    adj_list2=[]
    for i in range(5000):
        head=node(i)
        adj_list2.append(head)
    verts=list(np.arange(5000))

    edges2=[]
    for v in range (0,5000):    
        degree_v=np.random.randint(80,120)
        p=adj_list2[v]
        while(p.nex!=None):
            p=p.nex
        vert_ids=np.random.choice(verts,int(len(verts)/5),replace=False)
        for edge in range(0,len(vert_ids)):
            weight=np.random.randint(1,1000)
            w=vert_ids[edge]
            p.nex=node(w,weight)
            edges2.append([v,w,weight])
            p=p.nex

            q=adj_list2[w]
            while(q.nex!=None):
                q=q.nex
            q.nex=node(v,weight)
        #add one edge to next vertex to ensure graph is connected
        if v<4999:
            weight=np.random.randint(1,1000)
            p.nex=node(v+1,weight)
        verts.remove(v)
        if v%500==0:
            print(v,' of 5000')
    return adj_list2,edges2

Now let us implement a max heap, this will be used later for implementing a dijkstras style algorithm for routing. We see it being used for finding the optimum vertex in the fringe.

In [137]:
#now let us implement a max heap
#vertex ids
H=[]

#vertex values
D=list(np.zeros(5000))

In [138]:
def insert(H,D,vert_id,value):
    D[vert_id]=value
    n=len(H)
    n+=1
    H.append(vert_id)
    i=n-1
    #while there are more than 1 members, and element is greater than parent
    #push up new element while the max heap property is broken
    while(i>0 and D[H[i]]>D[H[math.floor(i/2)]]):
        #swap with parent
        temp=H[i]
        H[i]=H[math.floor(i/2)]
        H[math.floor(i/2)]=temp
        i=math.floor(i/2)
    return(H,D)

In [139]:
for i in range(10):
    H,D=insert(H,D,i,np.random.randint(1,10))

print(len(H),D[0:10])

10 [7, 1, 8, 6, 7, 6, 1, 2, 4, 4]


In [140]:
def maximum(H,D):
    return H[0],D[H[0]]

In [141]:
def delete(H,D,loc):
    H[loc]=H[-1]
    del H[-1]
    n=len(H)-1
    n-=1
    i=loc
    
    #first let us handle the case when the element to be deleted is not the head
    
    #first let us handle broken max heap property with parent
    if (i>0 and D[H[i]]>D[H[math.floor(i/2)]]):
        while (i>0 and D[H[i]]>D[H[math.floor(i/2)]]):
            temp=H[i]
            H[i]=H[math.floor(i/2)]
            H[math.floor(i/2)]=temp
            i=math.floor(i/2)
    #now let us handle broken property with children
    else:
        #while the max heap property is broken with children swap with larger of children
        while(2*i<=n and (D[H[i]]<D[H[2*i]] or D[H[i]]<D[H[2*i+1]])):
            if(2*i==n or D[H[2*i]]>D[H[2*i+1]]):
                temp=H[i]
                H[i]=H[2*i]
                H[2*i]=temp
                i=2*i
            else:
                temp=H[i]
                H[i]=H[2*i+1]
                H[2*i+1]=temp
                i=2*i+1
    return(H,D)

Let us check the heap functions. 

In [142]:
print(H,D[0:10])
for i in range(10):
    print(D[H[i]])
H,D=delete(H,D,2)
print(H,D[0:10])
for i in range(9):
    print(D[H[i]])

[2, 0, 4, 3, 8, 5, 6, 7, 1, 9] [7, 1, 8, 6, 7, 6, 1, 2, 4, 4]
8
7
7
6
4
6
1
2
1
4
[2, 0, 5, 3, 8, 9, 6, 7, 1] [7, 1, 8, 6, 7, 6, 1, 2, 4, 4]
8
7
6
6
4
4
1
2
1


Now we will try implementing some routing algorithms for maximum bandwidth path. For the first one we will use a modified version of djikstras algorithm and for the fringes we will not be using the heap structure we have developed so far.

In [143]:
def djikstra_noheap(adj_list,s,t):

    #let us first define the data structures which will be used here

    #status will be zero for unseen, one for in fringe and 2 for in tree
    #all vertices are initialized as unseen
    status=np.zeros(5000)

    #bandwidth of each vertex is the min bw of the path from the source to that vertex
    bw=np.ones(5000)
    bw=bw*10000

    #dad array to give the path to each vertex 
    dad=np.zeros(5000)

    #a datastructure to handle fringes
    F=[]

    # let us define source vertex as vertex 0
    S=adj_list[s]

    #let us define T vertex as last vertex
    T=adj_list[t]

    #initialize
    status[S.vert_id]=2

    start=time.time()

    # assign all vertices connected to s as fringes and update dad array
    p=S.nex
    while(p!=None):
        status[p.vert_id]=1
        dad[p.vert_id]=S.vert_id
        bw[p.vert_id]=min(bw[S.vert_id],p.weight)
        F.append(p.vert_id)
        p=p.nex
    #print(F)

    #now one at a time add the best vertex from the fringe into the intree. here the criteria for best is the one with
    #maximum bandwidth.then update the dad array, the bw array and also add new vertices into the fringe accordingly.

    while(len(F)!=0):
        max_F=0
        for i in range (len(F)):
            if F[i]>max_F:
                    max_F=i
        #F_bws=np.array(bw[F])
        #u=F[np.argmax(F_bws)]
        u=F[max_F]
        status[u]=2
        F.remove(u)

        #now let us visit all of u's edges
        p=adj_list[u].nex
        while(p!=None):
            v=p.vert_id
            # if vertex is unseen
            if status[v]==0:
                status[v]=1
                dad[v]=u
                bw[v]=min(bw[u],p.weight)
                F.append(v)
            #if vertex is a fringe and if we should update its bandwidth
            elif (status[v]==1 and min(bw[u],p.weight)>bw[v]):
                #status[v]=2
                dad[v]=u
                bw[v]=min(bw[u],p.weight)
            p=p.nex
    end=time.time()
    #print(end-start)
    return (end-start,dad)

In [144]:
adj_list,edges=graph_gen2()
te,dad=djikstra_noheap(adj_list,0,499)
print(te,len(edges))

0  of 5000
500  of 5000
1000  of 5000
1500  of 5000
2000  of 5000
2500  of 5000
3000  of 5000
3500  of 5000
4000  of 5000
4500  of 5000
6.506446123123169 2498500


In [145]:
adj_list,edges=graph_gen1()
te,dad=djikstra_noheap(adj_list,0,499)
print(te,len(edges))

0.8768455982208252 22412


In [131]:
w=dad[499]
while(w!=0):
    print (w)
    w=dad[int(w)]
print (w)

2623.0
0.0


In [132]:
te,dad=djikstra_heap(adj_list,0,499)
w=dad[499]
while(w!=0):
    print (w)
    w=dad[int(w)]
print (w)

654.0
4578.0
3947.0
418.0
2047.0
1964.0
0.0


Now let us try and implement the maximum bandwidth path using the heap that we have designed.

In [116]:
def djikstra_heap(adj_list,s,t):

    #let us first define the data structures which will be used here

    #status will be zero for unseen, one for in fringe and 2 for in tree
    #all vertices are initialized as unseen
    status=np.zeros(5000)

    #bandwidth of each vertex is the min bw of the path from the source to that vertex
    bw=np.ones(5000)
    bw=bw*10000

    #dad array to give the path to each vertex 
    dad=np.zeros(5000)

    #a datastructure to handle fringes by using heap
    F_H=[]
    F_count=0
    F=[]

    #vertex values
    D=list(np.zeros(5000))

    # let us define source vertex as vertex 0
    S=adj_list[0]

    #let us define T vertex as last vertex
    T=adj_list[-1]

    #initialize
    status[S.vert_id]=2

    start=time.time()
    # assign all vertices connected to s as fringes and update dad array
    p=S.nex
    while(p!=None):
        status[p.vert_id]=1
        dad[p.vert_id]=S.vert_id
        bw[p.vert_id]=min(bw[S.vert_id],p.weight)
        F_H,D=insert(F_H,D,p.vert_id,p.weight)
        F_count+=1
        F.append(p.vert_id)
        p=p.nex


    #now one at a time add the best vertex from the fringe into the intree. here the criteria for best is the one with
    #maximum bandwidth.then update the dad array, the bw array and also add new vertices into the fringe accordingly.
    
    while(F_count!=0):
        #pdb.set_trace()
        u,u_bw=maximum(F_H,D)
        status[u]=2
        F_H,D=delete(F_H,D,0)
        F_count-=1
        #F.remove(u)

        #now let us visit all of u's edges
        p=adj_list[u].nex
        
        while(p!=None):
            v=p.vert_id
            # if vertex is unseen
            if status[v]==0:
                status[v]=1
                dad[v]=u
                bw[v]=min(bw[u],p.weight)
                F_H,D=insert(F_H,D,p.vert_id,p.weight)
                #F.append(p.vert_id)
                F_count+=1
            #if vertex is a fringe and if we should update its bandwidth
            elif (status[v]==1 and min(bw[u],p.weight)>bw[v]):
                dad[v]=u
                bw[v]=min(bw[u],p.weight)
            p=p.nex
    end=time.time()
    #print(end-start)
    return(end-start,dad)

In [114]:
adj_list,edges=graph_gen2()

0  of 5000


KeyboardInterrupt: 

In [ ]:
te,dad=djikstra_heap(adj_list,0,4999)
print(te)

Now we can try implementing kruskals modification to solve the maximum bandwidth path

In [16]:
def find(v,p):
    #pdb.set_trace()
    #print(v,p[v])
    w=v
    while (p[int(w)]!=-1):
        w=p[int(w)]
    return int(w)

In [17]:
def union(r1,r2,dad,e,adj_listk,weight,rank):
    if (rank[r1]>rank[r2]):
        dad[r2]=int(r1)
    elif (rank[r2]>rank[r1]):
        dad[r1]=int(r2)
    else:
        dad[r1]=int(r2)
        rank[r2]+=1
    #add edge to graph
    p=adj_listk[e[0]]
    while(p.nex!=None):
        p=p.nex
    p.nex=node(e[1],weight)
    
    p=adj_listk[e[1]]
    while(p.nex!=None):
        p=p.nex
    p.nex=node(e[0],weight)
    
    return dad,rank,adj_listk

In [18]:
def dfs (adj_listk,dad,p,i,flag):
    #pdb.set_trace()
    if (p.vert_id ==4999 or dad[p.vert_id]!=0) and flag>0 and p.weight!=None:
        return dad
    else:
        flag+=1
        while (p.nex!=None):
            if dad[p.nex.vert_id]==0:
                dad[p.nex.vert_id]=i
                p=p.nex
                #print(p)
                dad=dfs(adj_listk,dad,adj_listk[p.vert_id],p.vert_id,flag)
            else:
                p=p.nex
    return dad

In [21]:
def kruskal(edges,s,t):
    #Now let us create an adjacency list with all the vertices in the graph
    adj_listk=[]
    for i in range(5000):
        head=node(i)
        adj_listk.append(head)
    start = time.time()

    #dad array
    dad=np.ones(5000)
    dad=dad*-1
    rank=np.zeros(5000)

    #all edges in edges1
    #first we want to heap sort the edges by their weights
    E_H=[]
    D=np.zeros(len(edges))

    for i in range (len(edges)):
        E_H,D=insert(E_H,D,i,edges[i][2])

    #pdb.set_trace()
    sorted_edges1=[]
    for j in range (len(edges)):
        sorted_edges1.append(maximum(E_H,D))
        E_H,D=delete(E_H,D,0)


    
    for i in range(len(edges)):
        e=(edges[sorted_edges1[i][0]][0],edges[sorted_edges1[i][0]][1])
        weight=edges[sorted_edges1[i][0]][2]
        r1=find(e[0],dad)
        r2=find(e[1],dad)
        if (r1!=r2):
            dad,rank,adj_listk=union(r1,r2,dad,e,adj_listk,weight,rank)
    
    
    p=adj_listk[0]
    dad=np.zeros(5000)
    dad[0]=-1
    
    dad=dfs(adj_listk,dad,p,s,0)
    end = time.time()
    #print(end-start)
    return (adj_listk,end-start)

In [22]:
adj_list,edges=graph_gen1()
adj_listk,te=kruskal(edges,0,4999)
print(te)

0.379986047744751


Now let us do dfs from s and find the st  p

In [119]:
time_dense={'Djikstra_noheap':[],'Djikstra_heap':[],'kruskal':[]}
time_sparse={'Djikstra_noheap':[],'Djikstra_heap':[],'kruskal':[]}
sts=[]
for i in range(5):
    #np.random.seed(i+2)
    st=np.random.choice(5000,size=10,replace=False)
    sts.append(st)
    adj_list1,edges1=graph_gen1()
    adj_list2,edges2=graph_gen2()
    for j in range (5):
        s=j*2
        t=j*2+1
        te,dad=djikstra_noheap(adj_list1,st[s],st[t])
        print(i,' Djikstra_noheap : ',te)
        time_sparse['Djikstra_noheap'].append(te)
        te,dad=djikstra_heap(adj_list1,st[s],st[t])
        print(i,' Djikstra_heap : ',te)
        time_sparse['Djikstra_heap'].append(te)
        adj_listk,te=kruskal(edges1,st[s],st[t])
        print(i,' kruskal : ',te)
        time_sparse['kruskal'].append(te)
        te,dad=djikstra_noheap(adj_list2,st[s],st[t])
        print(i,' Djikstra_noheap : ',te)
        time_dense['Djikstra_noheap'].append(te)
        te,dad=djikstra_heap(adj_list2,st[s],st[t])
        print(i,' Djikstra_heap : ',te)
        time_dense['Djikstra_heap'].append(te)
        adj_listk,te=kruskal(edges2,st[s],st[t])
        print(i,' kruskal : ',te)
        time_dense['kruskal'].append(te)


0  of 5000
500  of 5000
1000  of 5000
1500  of 5000
2000  of 5000
2500  of 5000
3000  of 5000
3500  of 5000
4000  of 5000
4500  of 5000
0  Djikstra_noheap :  0.9604141712188721
0  Djikstra_heap :  0.09506964683532715
0  kruskal :  0.36040496826171875
0  Djikstra_noheap :  6.524407386779785
0  Djikstra_heap :  5.355813026428223
0  kruskal :  59.06829476356506
0  Djikstra_noheap :  0.8230741024017334
0  Djikstra_heap :  0.09593701362609863
0  kruskal :  0.3598518371582031
0  Djikstra_noheap :  6.391582727432251
0  Djikstra_heap :  5.06118369102478
0  kruskal :  58.01395916938782
0  Djikstra_noheap :  1.0104858875274658
0  Djikstra_heap :  0.10253167152404785
0  kruskal :  0.3596971035003662
0  Djikstra_noheap :  6.3130786418914795
0  Djikstra_heap :  5.1317362785339355
0  kruskal :  58.185612201690674
0  Djikstra_noheap :  0.929145097732544
0  Djikstra_heap :  0.09604763984680176
0  kruskal :  0.37310242652893066
0  Djikstra_noheap :  5.98224401473999
0  Djikstra_heap :  5.13128232955932

KeyboardInterrupt: 